In [1]:
from cmr import CollectionQuery, GranuleQuery, ToolQuery, ServiceQuery, VariableQuery
import xarray as xr
import numpy as np
import pandas as pd

In [2]:
provider = 'GES_DISC'
api = CollectionQuery()
collections = api.provider(provider).get_all()

print('provider ' + provider + ' has ' + str(np.shape(collections)[0]) + ' to check')

provider GES_DISC has 1791 to check


In [3]:
# sad that because GES DISC didn't leverage opendap flags we have to use this
# silly way to identify those collections :( )

# should we add filter for just checking level 3/4 products? 
# for now will summarize by this info 

In [4]:
opendap_base_urls = []
short_names = []
versions = []
processing_levels = []
native_ids = []

for collection in collections:
    for element in collection['links']:
        if element['rel'] == 'http://esipfed.org/ns/fedsearch/1.1/service#':
            # this should return default url so can be on prem or cloud depending on granule
            opendap_base_urls.append(element['href'])
            processing_levels.append(collection['processing_level_id'])
            versions.append(collection['version_id'])
            short_names.append(collection['short_name'])
            native_ids.append(collection['id'])
            break

print('provider ' + provider + ' has ' + str(np.shape(short_names)[0]) + ' opendap collections to check')

provider GES_DISC has 1305 opendap collections to check


In [5]:
# get granule for each collection, try to open with xarray


In [6]:
api = GranuleQuery()
granule = api.short_name(short_names[0]).version(versions[0]).get(1)

In [7]:
message_out = []
short_names_out = []
versions_out = []
opendap_urls_out = []
processing_levels_out = []

for i in range(0,np.shape(short_names)[0]):
    print(i)
    granule = api.short_name(short_names[i]).version(versions[i]).get(1) 
    for element in granule[0]['links']:
        if 'title' in element:
            if element['title'] == 'The OPENDAP location for the granule. (GET DATA : OPENDAP DATA)':
                # this should return default url so can be on prem or cloud depending on granule
                opendap_url = (element['href'])
                opendap_urls_out.append(opendap_url)
                try:
                    xr.open_dataset(opendap_url)
                    message_out.append('success')
                    short_names_out.append(short_names[i])
                    versions_out.append(versions[i])
                    processing_levels_out.append(processing_levels[i])
                except Exception as e:
                    message_out.append(e)
                    short_names_out.append(short_names[i])
                    versions_out.append(versions[i])
                    processing_levels_out.append(processing_levels[i])

        #else:
            #message_out.append('no opendap url found')
    df_out = pd.DataFrame({'short_name':short_names_out,'version':versions_out,'processing_level':processing_levels_out,'message':message_out,'opendap_url':opendap_urls_out})
    df_out.to_csv('results_' + provider +  '.csv')


767
768
769
